# Make Netcdf Mask of Countries


In [44]:
# Download the shapefile fo a country from the web : 
# !pip install iso3166
from iso3166 import countries
#!pip install wget
import wget
import os
from netCDF4 import Dataset as NetCDFFile
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np 

In [65]:
# get your country's 3 letter  code: 
country= countries.get('Kazakhstan').alpha3

In [67]:
# Download the file 
!rm *.zip 
url=f"https://biogeo.ucdavis.edu/data/diva/adm/"+country+"_adm.zip"
wget.download(url, './shapefile.zip')


'./shapefile.zip'

In [70]:
# Unzip the zip file! 
!rm *_adm*.*
!rm *txt 
!unzip ./shapefile.zip

Archive:  ./shapefile.zip
 extracting: KAZ_adm0.cpg            
  inflating: KAZ_adm0.csv            
  inflating: KAZ_adm0.dbf            
  inflating: KAZ_adm0.prj            
  inflating: KAZ_adm0.shp            
  inflating: KAZ_adm0.shx            
 extracting: KAZ_adm1.cpg            
  inflating: KAZ_adm1.csv            
  inflating: KAZ_adm1.dbf            
  inflating: KAZ_adm1.prj            
  inflating: KAZ_adm1.shp            
  inflating: KAZ_adm1.shx            
 extracting: KAZ_adm2.cpg            
  inflating: KAZ_adm2.csv            
  inflating: KAZ_adm2.dbf            
  inflating: KAZ_adm2.prj            
  inflating: KAZ_adm2.shp            
  inflating: KAZ_adm2.shx            
  inflating: license.txt             


In [73]:
# Create a netcdf file with 0.1 deg resolution
!cdo -f nc -remapbil,r3600x1800 -setmissval,1e20 -topo topo_0.1x0.1deg_global.nc

cdo(1) setmissval: Process started
cdo(2) topo: Process started
cdo    remapbil: Bilinear weights from lonlat (720x360) to lonlat (3600x1800) grid
cdo    remapbil:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 91cdo(2) topo: 
cdo(1) setmissval: Processed 259200 values from 1 variable over 1 timestep.
cdo    remapbil: Processed 259200 values from 1 variable over 1 timestep [3.37s 1422MB].


In [74]:
# Download the ncl script 
url = "https://www.ncl.ucar.edu/Applications/Scripts/shapefile_utils.ncl"
wget.download(url, './shapefile_utils.ncl')

'./shapefile_utils (1).ncl'

In [75]:
# change the ncl code 
cmd = "sed -i -- 's/CCC/"+country+"/g' my_ncl_code.ncl"
os.system(cmd)
# run the ncl code : 
cmd='ncl my_ncl_code.ncl'
os.system(cmd)
# change back to default: 
cmd = "sed -i -- 's/"+country+"/CCC/g' my_ncl_code.ncl"
os.system(cmd)


0

## Now plot the netcdf file! 

In [76]:
# read the model data: 
nc = NetCDFFile(country+'_adm0_mask_array_0.1x0.1deg_global.nc')
lats = nc.variables['lat'][:]
lons = nc.variables['lon'][:]
mask = nc.variables['mask_array'][:]
nc.close()

In [ ]:
# plot the mask : 
fig = plt.figure('1')
fig.set_size_inches(14, 10)
my_map = Basemap(projection='cyl', 
                 llcrnrlat=-90, llcrnrlon=0,
                 urcrnrlat=90, urcrnrlon=360,
                 resolution='i', area_thresh=1.0)

my_map.fillcontinents(color='wheat', zorder=-1)
#my_map.drawcountries()
lons_m, lats_m = my_map(lons, lats)
# remove negative
masks = mask.data.astype(float)
masks[masks<0]=np.nan
# plot
my_map.pcolor(lons_m, lats_m, masks,vmin=-.0001, vmax=1,
              cmap='binary', zorder=1, alpha=1,shading='auto')
parallels = np.arange(-90.,90,20.)
# labels = [left,right,top,bottom]
my_map.drawparallels(parallels,labels=[False,True,True,False])
meridians = np.arange(0.,360.,20.)
my_map.drawmeridians(meridians,labels=[True,False,False,True])
plt.savefig("mask.png", bbox_inches = 'tight',
    pad_inches = 0);